In [1]:
from rich.console import Console
from rich.table import Table
import os
import math
import sys

In [2]:
sys.path.append("../src/")

In [14]:
from util.parse_time import parseTime, order_based_on_file_timestamp, find_matching_time

In [4]:
filesSatellite = order_based_on_file_timestamp(os.listdir('../data/preprocessed/satellite'))
filesRadar = order_based_on_file_timestamp(os.listdir('../data/preprocessed/radar/'))

In [19]:
lastRadar = parseTime(filesRadar[-1])
lastSatellite = parseTime(filesSatellite[-1])

lastDate = min(lastRadar, lastSatellite)

In [20]:
lastTimeMatch = find_matching_time(filesSatellite, lastDate)

In [21]:
newFilesSatellite = filesSatellite[0:lastTimeMatch]

In [22]:
filesSatelliteAmount = len(newFilesSatellite)
filesRadarAmount = len(filesRadar)

In [23]:
toRadarResolution = lambda sat: math.floor(sat * (3275 / filesSatelliteAmount))

In [24]:
trainSplit = math.floor(filesSatelliteAmount * 0.8)
valSplit = math.floor(filesSatelliteAmount * 0.1)
testSplit = math.floor(filesSatelliteAmount * 0.1)

In [25]:
table = Table(title="Files")

table.add_column("Start Date Sat", justify="right", style="cyan")
table.add_column("End Date Sat", justify="right", style="cyan")
table.add_column("Start Date Rad", justify="right", style="cyan")
table.add_column("End Date Rad", justify="right", style="cyan")
table.add_column("Partition", style="magenta")
table.add_column("Files Satellite", justify="right", style="green")
table.add_column("Files Radar", justify="right", style="green")

table.add_row(
    str(parseTime(newFilesSatellite[0])),
    str(parseTime(newFilesSatellite[-1])),
    str(parseTime(filesRadar[0])),
    str(parseTime(filesRadar[-1])),
    "All",
    str(filesSatelliteAmount),
    str(filesRadarAmount)
)

table.add_row(
    str(parseTime(newFilesSatellite[0])),
    str(parseTime(newFilesSatellite[trainSplit])),
    str(parseTime(filesRadar[toRadarResolution(0)])),
    str(parseTime(filesRadar[toRadarResolution(trainSplit)])),
    "Training",
    str(trainSplit),
    str(toRadarResolution(trainSplit))
)
table.add_row(
    str(parseTime(newFilesSatellite[trainSplit])),
    str(parseTime(newFilesSatellite[trainSplit + valSplit])),
        str(parseTime(filesRadar[toRadarResolution(trainSplit)])),
    str(parseTime(filesRadar[toRadarResolution(trainSplit + valSplit)])),
    "Validation",
    str(valSplit),
    str(toRadarResolution(valSplit))
)
table.add_row(
    str(parseTime(newFilesSatellite[trainSplit + valSplit])),
    str(parseTime(newFilesSatellite[trainSplit + valSplit + testSplit])),
        str(parseTime(filesRadar[toRadarResolution(trainSplit + valSplit)])),
    str(parseTime(filesRadar[toRadarResolution(trainSplit+valSplit+testSplit)])),
    "Testing",
    str(testSplit),
    str(toRadarResolution(testSplit))
)


console = Console()
console.print(table)

                                                       Files                                                       
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃                ┃                 ┃                ┃                 ┃            ┃          Files ┃             ┃
┃ Start Date Sat ┃    End Date Sat ┃ Start Date Rad ┃    End Date Rad ┃ Partition  ┃      Satellite ┃ Files Radar ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│     2023-04-01 │      2023-04-12 │     2023-04-01 │      2023-04-12 │ All        │           1086 │        3303 │
│       00:12:00 │        11:12:00 │       00:05:00 │        11:15:00 │            │                │             │
│     2023-04-01 │      2023-04-10 │     2023-04-01 │      2023-04-10 │ Training   │            868 │        2617 │
│       00:12:00 │        04:57:00 │       00:05:00 │        02:10:00 │            │                │             │
│     2023-04-10 │      2023-04-11 │     2023-04-10 │      2023-04-11 │ Validation │            108 │         325 │
│       04:57:00 │        07:57:00 │       02:10:00 │        05:20:00 │            │                │             │
│     2023-04-11 │      2023-04-12 │     2023-04-11 │      2023-04-12 │ Testing    │            108 │         325 │
│       07:57:00 │        10:57:00 │       05:20:00 │        08:25:00 │            │                │             │
└────────────────┴─────────────────┴────────────────┴─────────────────┴────────────┴────────────────┴─────────────┘